[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/claresinger/PySDM-examples/blob/ese134/PySDM_examples/Shima_et_al_2009/collisions_playground.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/claresinger/PySDM-examples/blob/ese134/PySDM_examples/Shima_et_al_2009/collisions_playground.ipynb)

#### Cloud Microphysics: Module 2
- Collisions and coalescence of cloud droplets
- Explore how 

Based on Fig. 2 from Shima et al. 2009 (Q. J. R. Meteorol. Soc. 135)  "_The super‐droplet method for the numerical simulation of clouds and precipitation: a particle‐based and probabilistic microphysics model coupled with a non‐hydrostatic model_"   
https://doi.org/10.1002/qj.441

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install atmos-cloud-sim-uj-utils
    from atmos_cloud_sim_uj_utils import pip_install_on_colab
    pip_install_on_colab('PySDM-examples')

In [2]:
from numpy import errstate
from IPython.display import display

from PySDM import Formulae
from PySDM.dynamics.collisions.collision_kernels import Golovin
from PySDM.initialisation import spectra
from PySDM.physics import si

from PySDM_examples.Shima_et_al_2009.spectrum_plotter import SpectrumPlotter
from PySDM_examples.Shima_et_al_2009.settings import Settings
from PySDM_examples.Shima_et_al_2009.example import run
from PySDM_examples.utils import widgets

In [3]:
progbar = widgets.IntProgress(min=0, max=100, description='%')

In [4]:
def demo(*, freezer, n_SD, b, r, smooth):
    frm = Formulae()
    with freezer:
        with errstate(all='raise'):
            n_step = 3600
            n_plot = 3
            settings = Settings(steps=[i * (n_step // n_plot) for i in range(n_plot + 1)])
            settings.n_sd = 2 ** n_SD
            settings.adaptive = True
            settings.dt = 10
            settings.kernel = Golovin(b=b / si.second)
            settings.X0 = frm.trivia.volume(radius=r * si.micrometres)
            settings.spectrum = spectra.Exponential(norm_factor=settings.norm_factor, scale=settings.X0)
            states, _ = run(settings, (widgets.ProgbarUpdater(progbar, settings.output_steps[-1]),))

        with errstate(invalid='ignore'):
            plotter = SpectrumPlotter(settings)
            plotter.smooth = smooth
            for step, state in states.items():
                plotter.plot(state, step * settings.dt)
            plotter.show()

In [6]:
n_SD = widgets.IntSlider(value=14, min=12, max=18, step=1, 
                         description='$log_2(n_{SD})$', continuous_update=False)
b = widgets.FloatSlider(value=1.5e3, min=1e3, max=2e3, step=1e2, 
                        description='$b$ in Golovin', continuous_update=False)
r = widgets.FloatSlider(value=30.531, min=20, max=40, step=1, 
                        description='$r$ in spectrum', continuous_update=False)
sliders = widgets.HBox([n_SD, b, r])

smooth = widgets.Checkbox(value=False, description='smooth plot')
options = [smooth]
boxes = widgets.HBox(options)
freezer = widgets.Freezer([n_SD, b, r])
self = widgets.interactive_output(demo, 
                                  {'freezer': freezer, 'n_SD': n_SD, 'b': b, 'r': r, 'smooth': smooth})

display(sliders, boxes, progbar, self)

IntProgress(value=100, description='%')

Output()